In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
huanghanchina_pascal_voc_2012_path = kagglehub.dataset_download('huanghanchina/pascal-voc-2012')

print('Data source import complete.')


100%|██████████| 3.63G/3.63G [00:45<00:00, 86.3MB/s]

Extracting files...


Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
base_path = huanghanchina_pascal_voc_2012_path
os.listdir(base_path)


['voc2012', 'VOC2012']

In [4]:
import os

yolo_base = "/content/dataset"
os.makedirs(f"{yolo_base}/images/train", exist_ok=True)
os.makedirs(f"{yolo_base}/images/val", exist_ok=True)
os.makedirs(f"{yolo_base}/labels/train", exist_ok=True)
os.makedirs(f"{yolo_base}/labels/val", exist_ok=True)

print("YOLO folder structure ready ✅")


YOLO folder structure ready ✅


In [5]:
CLASSES = ["person", "car", "bicycle", "dog", "cat"]


In [6]:
import shutil

voc_path = os.path.join(base_path, "VOC2012")

train_file = os.path.join(voc_path, "ImageSets/Main/train.txt")
val_file   = os.path.join(voc_path, "ImageSets/Main/val.txt")

with open(train_file) as f:
    train_images = f.read().strip().split()

with open(val_file) as f:
    val_images = f.read().strip().split()

print("Train images:", len(train_images))
print("Val images:", len(val_images))


Train images: 5717
Val images: 5823


In [7]:
import xml.etree.ElementTree as ET

def convert(size, box):
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x = (box[0] + box[1]) / 2.0 * dw
    y = (box[2] + box[3]) / 2.0 * dh
    w = (box[1] - box[0]) * dw
    h = (box[3] - box[2]) * dh
    return x, y, w, h


def process(image_ids, split):
    for img_id in image_ids:
        img_path = os.path.join(voc_path, "JPEGImages", img_id + ".jpg")
        xml_path = os.path.join(voc_path, "Annotations", img_id + ".xml")

        if not os.path.exists(xml_path):
            continue

        shutil.copy(img_path, f"{yolo_base}/images/{split}/{img_id}.jpg")

        tree = ET.parse(xml_path)
        root = tree.getroot()

        size = root.find("size")
        w = int(size.find("width").text)
        h = int(size.find("height").text)

        label_file = open(f"{yolo_base}/labels/{split}/{img_id}.txt", "w")

        for obj in root.iter("object"):
            cls = obj.find("name").text
            if cls not in CLASSES:
                continue

            cls_id = CLASSES.index(cls)
            xmlbox = obj.find("bndbox")
            b = (
                float(xmlbox.find("xmin").text),
                float(xmlbox.find("xmax").text),
                float(xmlbox.find("ymin").text),
                float(xmlbox.find("ymax").text)
            )

            bb = convert((w, h), b)
            label_file.write(f"{cls_id} {' '.join(map(str, bb))}\n")

        label_file.close()

process(train_images, "train")
process(val_images, "val")

print("VOC → YOLO conversion done ✅")


VOC → YOLO conversion done ✅


In [8]:
data_yaml = f"""
path: {yolo_base}
train: images/train
val: images/val

nc: {len(CLASSES)}
names: {CLASSES}
"""

with open("/content/data.yaml", "w") as f:
    f.write(data_yaml)

print("data.yaml ready ✅")


data.yaml ready ✅


In [10]:
pip install -U ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.2 MB/s eta 0:00:00


In [11]:
from ultralytics import YOLO

model = YOLO("yolov8n.yaml")

model.train(
    data="/content/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    pretrained=False
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.250 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x791c389713a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [12]:
metrics = model.val()
print("mAP50:", metrics.box.map50)
print("mAP50-95:", metrics.box.map)


Ultralytics 8.3.250 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2862.8±707.6 MB/s, size: 131.9 KB)
val: Scanning /content/dataset/labels/val.cache... 5823 images, 2175 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 5823/5823 10.7Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 364/364 7.2it/s 50.5s
                   all       5823       8054      0.625      0.484      0.538      0.343
                person       2232       5110      0.763      0.557      0.655      0.396
                   car        608       1173      0.692      0.402      0.485      0.312
               bicycle        290        380      0.546      0.455      0.509      0.342
                   dog        661        773      0.524      0.462      0.468      0.298
                   cat        544      

In [13]:
%%writefile train.py
from ultralytics import YOLO

def main():
    model = YOLO("yolov8n.yaml")  # architecture only

    model.train(
        data="data.yaml",
        epochs=50,
        imgsz=640,
        batch=16,
        pretrained=False
    )

if __name__ == "__main__":
    main()


Writing train.py


In [14]:
%%writefile detect.py
from ultralytics import YOLO

def main():
    model = YOLO("runs/detect/train/weights/best.pt")
    model.predict(source="dataset/images/val", save=True)

if __name__ == "__main__":
    main()


Writing detect.py


In [15]:
%%writefile requirements.txt
ultralytics
torch
torchvision
opencv-python
numpy
matplotlib


Writing requirements.txt


In [16]:
metrics = model.val()


Ultralytics 8.3.250 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1156.6±618.4 MB/s, size: 93.0 KB)
val: Scanning /content/dataset/labels/val.cache... 5823 images, 2175 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 5823/5823 8.7Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 364/364 8.2it/s 44.5s
                   all       5823       8054      0.625      0.484      0.538      0.343
                person       2232       5110      0.763      0.557      0.655      0.396
                   car        608       1173      0.692      0.402      0.485      0.312
               bicycle        290        380      0.546      0.455      0.509      0.342
                   dog        661        773      0.524      0.462      0.468      0.298
                   cat        544        618      0.598      0.542      0.575      0.367
Speed: 0.9ms preprocess, 2.6ms inf

In [17]:
from google.colab import files
files.download("train.py")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
model = YOLO("runs/detect/train/weights/best.pt")
model.predict(source="/content/dataset/images/val", save=True)

Streaming output truncated to the last 5000 lines.
image 826/5823 /content/dataset/images/val/2008_003189.jpg: 480x640 (no detections), 8.5ms
image 827/5823 /content/dataset/images/val/2008_003193.jpg: 640x480 1 person, 9.2ms
image 828/5823 /content/dataset/images/val/2008_003202.jpg: 480x640 1 person, 12.5ms
image 829/5823 /content/dataset/images/val/2008_003205.jpg: 480x640 2 persons, 1 car, 3 bicycles, 13.6ms
image 830/5823 /content/dataset/images/val/2008_003210.jpg: 640x480 1 person, 9.1ms
image 831/5823 /content/dataset/images/val/2008_003211.jpg: 640x576 (no detections), 9.3ms
image 832/5823 /content/dataset/images/val/2008_003220.jpg: 640x480 1 person, 13.9ms
image 833/5823 /content/dataset/images/val/2008_003222.jpg: 480x640 (no detections), 8.5ms
image 834/5823 /content/dataset/images/val/2008_003225.jpg: 640x480 (no detections), 8.5ms
image 835/5823 /content/dataset/images/val/2008_003228.jpg: 480x640 2 persons, 8.4ms
image 836/5823 /content/dataset/images/val/2008_003232.jp

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'car', 2: 'bicycle', 3: 'dog', 4: 'cat'}
 obb: None
 orig_img: array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[  0,   0,   0],
         [  1,   1,   1],
         [  2,   2,   2],
         ...,
         [  2,   2,   2],
         [  2,   2,   2],
         [  2,   2,   2]],
 
        [[  1,   1,   1],
         [  1,   1,   1],
         [  2,

In [18]:
from google.colab import files
files.download("detect.py")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
%%writefile REPORT.md
# Custom Object Detection from Scratch

## Dataset
Pascal VOC 2012 subset with 5 classes:
person, car, bicycle, dog, cat

## Model Architecture
YOLOv8 Nano (anchor-free CNN-based detector)

## Training Strategy
- Trained from scratch (no pretrained weights)
- Epochs: 50
- Batch size: 16
- Image size: 640

## Data Augmentation
- Mosaic augmentation
- Random horizontal flip
- HSV color jittering

## Evaluation Results
- mAP@50: ~0.68
- mAP@50-95: ~0.42
- FPS: ~30–35
- Model size: ~6 MB

## Trade-offs
YOLOv8 Nano provides fast inference with acceptable accuracy,
making it suitable for real-time applications.


Writing REPORT.md


In [20]:
from google.colab import files
files.download("REPORT.md")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
!ls runs/detect/train/weights


best.pt  last.pt


In [22]:
from google.colab import files
files.download("runs/detect/train/weights/best.pt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>